In [2]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

## Low Rank Approximation (PCA)

- Many data sets come in the form of a matrix.
- Think of $A$ as a stack of $n$ data points in $\mathbb{R}^d$.
- For example, $A$ can be a customer-product matrix, where each entry $A_{ij}$ refers to the number of times customer $i$ purchased item $j$.
- A data matrix $A$ is typically well-approximated by a low rank matrix, that is
$$
A = A_{approx} + E,
$$
where $rank(A_{approx})=k$ and $\|E\|_F$ is close to zero. (The *Frobenius norm* $\|\cdot\|_F$ is computed by summing up the square of its entries).
- Compare to $A$, $A_{approx}$ is easier to store and its values are more interpretable.

**Problem:** Given a matrix $A\in\mathbb{R}^{n\times d}$ and a $k\ll d<n$, find a $k$-dimensional subspace $S$ such that $\|A - \Pi_SA\|_F^2$ is minimized.

**Traiditional solution:** Truncated singular value decomposition (Truncated SVD)
Any matrix $A$ can be written as $A = U\cdot \Sigma \cdot V$, where
- Matrix $U$ has orthonormal column vectors
- Matrix $\Sigma$ is a diagonal matrix with non-increasing positive entries down the diagonal
- Matrix $V$ has othonormal row vectors

The best rank-$k$ approximation to $A$ is $A_k=U_k\cdot\Sigma_k\cdot V_k$, where
- Matrix $U_k$ consists of the first $k$ columns of $U$.
- Matrix $V_k$ consists of the first $k$ rows of $V$. (They are called the **top $k$ principal components**.)
- Matrix $\Sigma_k$ consists of the first $k$ diagonal elements of $\Sigma$.

Equivalently speaking, $A_k = argmin_{rank(B)=k}\|A - B\|_F$. The **main problem with the SVD approch** is its high computational complexity ($O(nd^2)$). Moreover, this method requires loading the entire matrix to the memory.

### Randomized Low Rank Approximation
**Goal:** Find a method that uses less than $nd^2$ time to output a rank-$k$ matrix $A'$ such that
$$
\|A - A'\|_F \le (1+\epsilon)\|A - A_k\|_F.
$$

**Existing literature:**
- [Clarkson-Woodruff 13]: near linear time in input sparsity
- [Ghashami-Liberty-Phillips-Woodruff 16]: streaming algorithms

**Main idea:**
- Construct a $k/\epsilon\times n$ subspace embedding matrix $S$ (such as CountSketch matrix)
- Compute $SA$, which forms a compact representation of rows of $A$.
- Project rows of $A$ onto $SA$.
- Find the best rank-$k$ approximation to points inside of $SA$. (Since $SA$ is a much smaller matrix, low-rank approximation of $SA$ is much easier to solve.)

### Choice of embedding matrix $S$

**Goal:** $S$ should be easy to construct, and the computation of $SA$ should require less than $ndk$ time.

A practical choice is the **CountSketch** matrix, where each column contains exactly one $\pm 1$ at a random row. For example:

\begin{equation*}
S =  
\begin{pmatrix}
0 & 0 & 1 & 0 & 0 & 1 & 0 & 0\\
1 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & -1& 1 & 0 & -1& 0\\
0 & 1 & 0 & 0 & 0 & 0 & 0 & 1
\end{pmatrix}
\end{equation*}

Because of the special structure of $S$, to compute $SA$,one only need to add $A_{(i)}$ to the result for each row $A_{(i)}$ of matrix $A$.

In [79]:
# An example of low-rank approximation using CountSketch matrix

# construct input matrix $A$
def input_mat(n, d, r, seed=42):
    """
    Generate an n*d random matrix with approximate rank r.
    
    Require: n > d > r > 0.
    """
    np.random.seed(seed)
    M = np.random.rand(n, d)
    U, S, V = np.linalg.svd(M, full_matrices=False)
    diagonal = np.hstack([np.linspace(1, 10, r), np.zeros(d - r)])
    new_S = np.diag(diagonal)
    return U.dot(new_S).dot(V)

n = 200
d = 100
k = 10
A = input_mat(n, d, k)
A.shape

(200, 100)

In [80]:
# Step 1: 
# Generate a CountSketch matrix S, represented by two lists h and sigma, where h stores the row number of 
# the non-zero entry on each column, and sigma stores its value.
def count_sketch(m, n, seed=42):
    """
    Generate an m*n CountSketch matrix
    
    Output:
    size: size of the matrix S
    h: a list storing the row number of the non-zero entry on each column
    sigma: a list storing the value of the non-zero entry on each column
    """
    np.random.seed(seed)
    ary = np.arange(m)
    size = [m, n]
    h = np.random.choice(ary, n)
    sigma = np.random.choice([-1, 1], n)
    return size, h, sigma
    
m = 20
S_size, S_h, S_sigma = count_sketch(m, n)
S_size

[20, 200]

In [81]:
# Step 2:
# Perform fast matrix multiplication SA.
def count_sketch_multi(S_size, S_h, S_sigma, A):
    """
    Multiply S and A.
    """
    m, n = S_size
    d = A.shape[1]
    result = np.zeros([m, d])
    for i in range(n):
        result[S_h[i], :] += S_sigma[i] * A[i, :]
    return result
SA = count_sketch_multi(S_size, S_h, S_sigma, A)
SA

array([[-0.53440505, -0.23080665,  0.7119568 , ..., -0.63387905,
         0.05230352,  0.25924817],
       [-0.72076792, -0.20804511,  0.43338964, ...,  0.43135791,
         0.12176035,  0.10669987],
       [-0.2393303 ,  0.17371414,  0.53481105, ..., -0.22615432,
         0.43641781, -0.20194093],
       ...,
       [-0.33691237, -0.01925639, -0.20071573, ...,  0.64413153,
        -0.15210403,  0.26085918],
       [ 0.152788  , -0.21561768,  0.53742216, ...,  0.09351726,
         0.10495453,  0.3206282 ],
       [-0.36520901,  0.2183839 , -0.23870602, ..., -0.92452083,
         0.012926  , -0.76878233]])

In [85]:
# Step 3:
# Project each of the rows onto SA.
def approx_embedding(SA, A, m2=None):
    """
    Compute an approximate rank-k embedding X of A onto SA, that is:
    a) rank(X) = k
    b) X * SA is close to A.
    """
    m, d = SA.shape
    
    if m2 is None:
        m2 = m
    R_size, R_h, R_sigma = count_sketch(m2, d)
    SAR = count_sketch_multi(R_size, R_h, R_sigma, SA.T).T
    AR = count_sketch_multi(R_size, R_h, R_sigma, A.T).T
    sol = np.linalg.lstsq(SAR.T, AR.T, rcond=None)
    return sol[0].T

def exact_embedding(SA, A):
    sol = np.linalg.lstsq(SA.T, A.T, rcond=None)
    return sol[0].T    

X1 = approx_embedding(SA, A)
X2 = exact_embedding(SA, A) # for comparison purpose
print(np.linalg.norm(X1 - X2)) # X2 should be close to X1

1.9094481481462404e-14


In [87]:
# Step 4: Find the best rank-k approximation of projected points inside of rowspace SA
def low_rank_approx(X, k):
    n, d = X.shape
    U, S, V = np.linalg.svd(X)
    Uk = U[:, :k]
    Sigmak = np.diag(S[:k])
    Vk = V[:k, :]
    return Uk.dot(Sigmak).dot(Vk)
X_approx = low_rank_approx(X1, k)
print(np.linalg.norm(X1 - X_approx))

1.0457730711665079e-14


In [89]:
# Verify that A is closely approximated by the product of X_approx and SA
print('Low rank approximation error:', np.linalg.norm(A - X_approx.dot(SA)))
print('size of A:', A.shape)
print('size of X_approx', X_approx.shape, ' | size of SA:', SA.shape)

Low rank approximation error: 9.940630662437842e-14
size of A: (200, 100)
size of X_approx (200, 20)  | size of SA: (20, 100)


### TODO: Test on real data

### Related Problems
- **Column subset selection (feature selection, CX, CUR)**: $\Pi$ consists of columns of $M$. ([Friez-Kannan-Vempala 98], [Drineas-Mahoney-Muthukrishnan 06], [Deshpande-Vempala 06])

### Online Algorithms

- Columns arrive one by one
- Need to add to $S$ or ignore
- Complete with best $S$ in hindsight (in approximation error)

**Online PCA:** [Boutsidis-Garber-Karnin-Liberty 15] additive approximation $\epsilon\|U\|_F^2$ using $k\cdot polylog(n)$ dimensions.

**Online CUR:** [Bhaskara 18] $(1+\epsilon)$ relative error using $k\cdot polylog(n/\epsilon)$ columns using adaptive sampling, if a) we have a $poly(n)$ approximation of OPT, and $\sigma^2_{max}/OPT$ is at most $poly(n, d)$.

**Online k-means:** [Meyerson 01] objective: small number of clusters, near-optimal centers

### Constructing sampling distribution

**Assumptions (not essential):**
- OPT ($=\|U - U_k\|_F$) is known 
- $\|U\|_2$ is known

**"Ideal" algorithm:**
- Select a column with probability proportional to how much it can reduce the objective function:
$$
p_i := Pr(u_i\rightarrow S) = \min\{1, k\cdot\frac{\|\Pi_S^\perp u_i\|_2^2}{OPT}\}
$$
- Difficult to analyze
- Observation: it is impossible to have many $p_i$ to be close to 1 (it means $u_i$ has a large component perpendicular to previous chosen columns). Otherwise $U$ is not numerial low rank (Geometric lemma).